In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np


transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

mnist_traindataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, 
                                                                                transform=transform)
mnist_testdataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, 
                                                                                transform=transform)

0it [00:00, ?it/s]

9920512it [00:01, 8682645.67it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 122323.01it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


1654784it [00:00, 2123682.24it/s]                            
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


8192it [00:00, 49419.84it/s]            


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [0]:
N_noise = batch_size = 128
epochs = 200

cuda_available = False
if torch.cuda.is_available():
    cuda_available = True

In [0]:
class Discriminator(torch.nn.Module):
    def __init__(self, input_node_size, hidden_node_size, output_node_size):
        """
        In case of P-MNIST
        ==============
        input_node_size: 784
        hidden_node_size: 256
        output_node_size: 1

        other cases
        ===============
        """
        super(Discriminator, self).__init__()
        fc1 = torch.nn.Linear(input_node_size, hidden_node_size)
        fc2 = torch.nn.Linear(hidden_node_size, hidden_node_size)
        fc3 = torch.nn.Linear(hidden_node_size, output_node_size)


        self.network = torch.nn.Sequential(
            fc1,
            torch.nn.ReLU(),
            fc2,
            torch.nn.ReLU(),
            fc3
        )

        if cuda_available:
            self.network = self.network.cuda()

    def forward(self, x):
        return torch.sigmoid(self.network(x))
    
    
class Generator(torch.nn.Module):
    def __init__(self, input_node_size, hidden_node_size, output_node_size):
        """
        In case of P-MNIST
        ==========
        input_node_size: N_noise
        hidden_node_size: 256
        output_node_size: 784

        other cases
        ===========
        """
        super(Generator, self).__init__()
        fc1 = torch.nn.Linear(input_node_size, hidden_node_size)
        fc2 = torch.nn.Linear(hidden_node_size, hidden_node_size)
        fc3 = torch.nn.Linear(hidden_node_size, output_node_size)

        self.network = torch.nn.Sequential(
            fc1,
            torch.nn.ReLU(),
            fc2,
            torch.nn.ReLU(),
            fc3
        )

        if cuda_available:
            self.network = self.network.cuda()

    def forward(self, x):
        return torch.sigmoid(self.network(x))

In [0]:
def GAN_training(dataloader, disc, gen, disc_optim, gen_optim):
    running_loss = 0.0
    for i, data in enumerate(dataloader):
        x, y = data
        x = x.view(-1, 784)
        b_size = x.shape[0]
        if cuda_available:
            x = x.cuda()
            y = y.cuda()

        noise = torch.FloatTensor(b_size, N_noise).uniform_(-1, 1)
        if cuda_available:
            noise = noise.cuda()

        # Learn Discriminator 
        gen_image = gen(noise)
        log_original_image_scores = torch.log(disc(x)) + torch.log(1 - disc(gen_image))
        
        disc_loss = -torch.sum(log_original_image_scores) / b_size
        disc_loss.backward()
        disc_optim.step()
        
        # Learn Generator
        gen_image = gen(noise)
        log_gen_image_scores = torch.log(disc(gen_image))
        
        gen_loss = -torch.sum(log_gen_image_scores) / b_size
        gen_loss.backward()
        gen_optim.step()

In [0]:
traindataloader = torch.utils.data.DataLoader(mnist_traindataset, batch_size=batch_size, shuffle=True)
testdataloader = torch.utils.data.DataLoader(mnist_testdataset, batch_size=batch_size, shuffle=True)

disc = Discriminator(784, 256, 1)
gen = Generator(N_noise, 256, 784)
disc_optim = torch.optim.Adam(disc.parameters())
gen_optim = torch.optim.Adam(gen.parameters())

for epoch in range(epochs):
    GAN_training(traindataloader, disc, gen, disc_optim, gen_optim)
    print(epoch)

    if epoch % 10 == 9:
        random_noise = torch.FloatTensor(25, N_noise).uniform_(-1, 1)
        if cuda_available:
            random_noise = random_noise.cuda()

        generated = gen(random_noise)
        fig, ax = plt.subplots(1, 25, figsize=(25, 1))
        for i in range(25):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(generated[i].cpu().detach().numpy().astype(int), (28, 28)))
        plt.savefig('samples_ex/{}.png'.format(str(epoch).zfill(3), bbox_inches='tight'))
        plt.close(fig)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
